# About
## 🎞️ Video Maker from NASA EPIC Images

This function generates a time-lapse video using a sequence of Earth observation images from NASA's [EPIC (Earth Polychromatic Imaging Camera)](https://epic.gsfc.nasa.gov/about/api) archive.

It takes a list of image identifiers, overlays each frame with a timestamp watermark, and stitches them into a smooth `.mp4` video.

### 🔧 Function Overview

```python
video_maker(
    output_video="earth_timelapse.mp4",
    duration=10,                        # duration in seconds
    list_images=[...],                  # list of EPIC image identifiers
    PATH_IMAGE="path/to/images/"        # folder containing .png images
)

# Environment

**Make you to use the requirements.txt in your environment**

In [ ]:
import cv2
import os
import pandas as pd

# CONSTANS and Variables

In [ ]:
# Read file to get identifiers
PATH_FILE = "ADD_THE_PATH_TO_SAVE_AN_EXCEL_FILE" # eg: 'your_path/your_file.xlsx' the same file you saved in API_NASA_BLUE_MARBLE.ipynb
MAIN_PATH = "ADD_THE_PATH_TO_DOWNLOAD_IMAGES"    # The path where you saved all the immages, the same of API_NASA_BLUE_MARBLE.ipynb
output_video = 'output_video.mp4'

# Functions

In [ ]:
def video_maker(output_video, duration, list_images, PATH_IMAGE=PATH_IMAGE):
    
    # Determine FPS
    fps = len(list_images) / duration

    # First image (to get size)
    first_image_path = os.path.join(PATH_IMAGE, f"{list_images[0]}.png")
    first_frame = cv2.imread(first_image_path)
    if first_frame is None:
        raise FileNotFoundError(f"First image not found: {first_image_path}")
    frame_height, frame_width = first_frame.shape[:2]

    # Video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_video, fourcc, fps, (frame_width, frame_height))

    # Font config
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_thickness = 2
    font_color = (255, 255, 255)  # White
    bg_color = (0, 0, 0)  # Black background for readability
    missing_keys = []
    # Process images
    for key in list_images:
        img_path = os.path.join(PATH_IMAGE, f"{key}.png")
        frame = cv2.imread(img_path)
        if frame is None:
            print(f"⚠️ Image not found: {img_path}")
            missing_keys.append(key)
            continue

        resized = cv2.resize(frame, (frame_width, frame_height))

        # Watermark text (timestamp)
        text = str(key)
        text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
        text_width, text_height = text_size

        # Position (bottom-right corner with padding)
        x = frame_width - text_width - 20
        y = frame_height - 20

        # Draw background rectangle for text
        cv2.rectangle(resized, (x - 5, y - text_height - 5), (x + text_width + 5, y + 5), bg_color, -1)
        # Draw text
        cv2.putText(resized, text, (x, y), font, font_scale, font_color, font_thickness, cv2.LINE_AA)

        video_writer.write(resized)

    video_writer.release()
    print("🎬 Video successfully saved as:", output_video)

# Read information to use

In [ ]:
df = pd.read_excel(PATH_FILE)
# Reverse the df > 2015-2025
df_use = df[::-1]
ordered_dict_keys = df_use['identifier'].to_list()

# Code Execution

In [ ]:
if __name__ == "__main__":    
    video_maker('output_example.mp4', 1200, ordered_dict_keys, PATH_IMAGE=PATH_IMAGE)